## Question 1: **Answers**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(data):
    try:
        data = pd.read_csv(data)
        # Connect to SQL and create a database
        engine = sqlite3.connect('customers_records.db')
        # Creating a table called 'purchases'
        data.to_sql('purchases', con=engine, index = False)
        engine.close()
    except Exception as e:
        print(f"Error: {e}")

data = "/content/customer_purchases.csv"
add_data_to_database(data)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///customers_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

#### 1. View the first five rows of the table?¶

In [6]:
%%sql
SELECT * FROM purchases
LIMIT 5;

 * sqlite:///customers_records.db
Done.


customer_name,product_name,purchase_type,amount_paid,date_of_purchase
Juliette Martin,Laptop,Walk-in Store,1200,2023-12-23
Lena Schneider,Mouse,Online,20,2023-12-24
Anna Nowicka,Ink Cartridges,Walk-in Store,50,2023-12-26
Alejandro Gonzalez,Monitor,Online,150,2023-12-26
Max Schmidt,Headphones,Walk-in Store,100,2023-12-27


## Question 2: **Answers**

In [7]:
%%sql
SELECT COUNT(*) AS count_of_rows_in_table
FROM purchases;

 * sqlite:///customers_records.db
Done.


count_of_rows_in_table
29


## Questions 3: **Answers**

In [8]:
%%sql
SELECT
    customer_name,
    printf('%20s', COUNT(DISTINCT(product_name)))
    AS unique_products_bought
FROM purchases
WHERE customer_name = 'Andrea Esposito';

 * sqlite:///customers_records.db
Done.


customer_name,unique_products_bought
Andrea Esposito,2


## Questions 4: **Answers**

In [9]:
%%sql
SELECT
    product_name,
    SUM(amount_paid) AS total_amount_brought_in
FROM purchases
GROUP BY product_name
HAVING total_amount_brought_in < 30
ORDER BY amount_paid ASC;

 * sqlite:///customers_records.db
Done.


product_name,total_amount_brought_in
Screen Protector,10
Stylus,10
Keyboard Cover,15
Mousepad,15
Mouse,20
Water Bottle,20


## Questions 5: **Answers**

In [10]:
%%sql
SELECT
    product_name,
    printf('%20s', COUNT(purchase_type)) AS count_online_purchases
FROM purchases
WHERE product_name = 'Charging Cable'
      AND purchase_type = 'Online';

 * sqlite:///customers_records.db
Done.


product_name,count_online_purchases
Charging Cable,2


## Question 6: **Answers**

In [11]:
%%sql
SELECT
    customer_name,
    CASE
        WHEN
            product_name = 'Smartwatch' AND
            purchase_type = 'Online' AND
            -- Checking if purchase with 30 days
            JULIANDAY('2024-05-20') - JULIANDAY(date_of_purchase) <= 30
            THEN 'Eligible for Refund'
        ELSE 'Not Eligible for Refund'
    END AS refund_status
FROM purchases
WHERE customer_name = 'Lorenzo Rossi';

 * sqlite:///customers_records.db
Done.


customer_name,refund_status
Lorenzo Rossi,Not Eligible for Refund


## Question 7: **Answers**

In [12]:
%%sql
SELECT COUNT(customer_name) AS customers_with_multiple_buys
FROM
    (SELECT
        customer_name,
        COUNT(product_name) AS product_bought
    FROM purchases
    GROUP BY customer_name
    HAVING product_bought > 1) AS customer_count;

 * sqlite:///customers_records.db
Done.


customers_with_multiple_buys
4


In [13]:
# %%sql
# DROP TABLE purchases;